In [0]:
!pip install tensorflow==1.14.0

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from PIL import Image

In [4]:
from google.colab import drive #STU NAPERVILLE ACCOUNTS DON'T WORK - USE YOUR PERSONAL GOOGLE ACCOUNTS
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#FORMATTING DATA SET (not tested)
#https://medium.com/@muskulpesent/create-numpy-array-of-images-fecb4e514c4b&sa=D&source=hangouts&ust=1582390259517000&usg=AFQjCNFIlPagqjM75QyaHiorDWqaazaL5g
train_images = []
train_labels = []
test_images = []
test_labels = []

#takes an exorbant amount of time to upload; may need to consider dataset pipelines or generators

files = glob.glob("/content/drive/My Drive/ROADREADER DATASETS/training/*.png")
for myFile in files:
  print(myFile)
  #image = cv2.imread(myFile)
  image = Image.open(myFile)
  image.load()
  image = np.asarray(image, dtype=np.float32)/255.0;
  train_images.append(image)
  train_labels.append(image[0])
train_labels = np.array(train_labels, dtype='float64')

files2 = glob.glob("/content/drive/My Drive/ROADREADER DATASETS/tests/*.png")
for myFile in files2:
  print(myFile)
  #image = cv2.imread(myFile)
  image = Image.open(myFile)
  image.load()
  image = np.asarray(image, dtype=np.float32)/255.0;
  test_images.append(image)
  test_labels.append(image[0])
test_labels = np.array(test_labels, dtype='float64')

trainingMap = dict(zip(train_images, train_labels))
testingMap = dict(zip(test_images, test_labels))

#mnist = keras.datasets.mnist
#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
print("shape before: " + str(np.shape(train_images)))
train_images = np.reshape(train_images, (-1, 300, 400, 3))
print("shape after: " + str(np.shape(train_images)))
test_images = np.reshape(test_images, (-1, 300, 400, 3))

shape before: (1187, 300, 400, 3)
shape after: (1187, 300, 400, 3)


In [3]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=(-1, 1))
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/training",
        target_size=(300, 400),
        batch_size=32, 
        color_mode="rgb",
        class_mode="sparse",
        shuffle=True)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=(-1, 1))
test_generator = test_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/tests",
        target_size=(300, 400),
        batch_size=1, 
        color_mode="rgb",
        class_mode=None,
        shuffle=False)

Found 1196 images belonging to 2 classes.
Found 204 images belonging to 2 classes.


In [4]:
model = keras.Sequential([
  layers.Conv2D(14, 3, activation='relu', input_shape=(300,400,3)),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(28, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(42, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Flatten(),
  layers.Dense(20000, activation='relu'),
  layers.Dense(200, activation='relu'),
  layers.Dense(2, activation='softmax'), #classification layer; 0 for none 1 for stop
])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 398, 14)      392       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 199, 14)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 197, 28)      3556      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 99, 28)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 97, 42)        10626     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 49, 42)        0         
____

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [1]:
#model.fit(x=train_images,y=train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=128)

model.fit_generator(
        train_generator,
        steps_per_epoch=128,
        epochs=10,
        validation_data=test_generator,
        validation_steps=32,
        workers=1,
        use_multiprocessing=False,
        max_queue_size=10)

'''
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)
                    '''

model.save_weights('/content/model.h5')

NameError: ignored

In [0]:
predictions = np.argmax(model.predict(test_images), axis=1)
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.reshape(test_images[i], (28,28)), cmap=plt.cm.binary)
    plt.xlabel(predictions[i])
plt.show()